In [1]:
import os
import cv2
import mediapipe
import numpy as np
import os

In [2]:
dim_x = 500
dim_y = 500

In [3]:
y_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
count_labels = len(y_labels)

n_cat = 1000
count = 0

In [12]:
y_labels[11]

'L'

In [4]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.7
color = (0, 0, 255)
thickness = 1

## Creating the Folders

path = r'datasets'
os.mkdir(path)
for label in y_labels:
    dir_path = os.path.join(path, label)
    os.mkdir(dir_path)

## Status:
1. collecting
2. paused

In [8]:
cap = cv2.VideoCapture(2)
sample = cv2.imread("ASL_Alphabet.jpg")
cv2.imshow("sample", sample)

status = "idle"
count = 0
curr_cat = int(11)
max_samples = 3000

frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frameWidth = frameHeight

drawingModule = mediapipe.solutions.drawing_utils
handsModule = mediapipe.solutions.hands

path = r"datasets"

with handsModule.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2) as hands:
    while (curr_cat < len(y_labels)):
        
        dir_path = os.path.join(path, y_labels[curr_cat])
        
        ret, frame = cap.read()
        frame = frame[0:frameHeight, 0:frameHeight]
        raw = frame.copy()

        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img = cv2.putText(frame, 'Sample Data Collector', (30,30), font, 1, (255,0,0), 1, cv2.LINE_AA)
        points = []
        if results.multi_hand_landmarks != None:
            for handLandmarks in results.multi_hand_landmarks:
                for point in handsModule.HandLandmark:
                    normalizedLandmark = handLandmarks.landmark[point]
                    pixelCoordinatesLandmark = drawingModule._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, frameWidth, frameHeight)
                    #print(pixelCoordinatesLandmark)
                    cv2.circle(frame, pixelCoordinatesLandmark, 4, (0, 255, 0), -1)
                    points.append(pixelCoordinatesLandmark)
            if(len(points) == 21):
                x_data = np.array(points)
                img = cv2.putText(frame, 'Hand Detected', (30,60), font, fontScale, color, thickness, cv2.LINE_AA)
                img = cv2.putText(frame, 'Category : '+y_labels[curr_cat], (30,90), font, fontScale, color, thickness, cv2.LINE_AA)
                img = cv2.putText(frame, 'Data point : '+str(count), (30,120), font, fontScale, color, thickness, cv2.LINE_AA)
                
                
                if(status == 'collecting' and count < max_samples):
                    count += 1
                    img_path = os.path.join(dir_path, str(count)+".jpg")
                    cv2.imwrite(img_path, raw) #save img
                    
                elif(count == max_samples):
                    count = 0
                    status = "idle"
                    curr_cat += 1
        
            elif(len(points) == 42):
                img = cv2.putText(frame, 'Please use only one hand', (30,60), font, fontScale, color, thickness, cv2.LINE_AA)
        else:
                img = cv2.putText(frame, 'Hand not detected', (30,60), font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow('Test hand', img)
        
        key = cv2.waitKey(1)
        if(key == 27 or key == ord('q')):
            break
        elif(key == ord('s') or key == ord('S')):
            cv2.waitKey(100)
            status = 'collecting'
            print(status)
        elif(key == ord('p') or key == ord('P')):
            status = 'paused'
            print(status)
        elif(key == ord('r') or key == ord('R')):
            status = "restart"
            print("restart??")
            count = 0
            status = "idle"
            print(status)
            #if in middle of acquizitaion, delete and retake, else delete old record
            #do somethinf
cv2.destroyAllWindows()
cap.release()

<ipython-input-8-2f42657eeedf>:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_data = np.array(points)


collecting
collecting
collecting
collecting
collecting
collecting
collecting
collecting


In [30]:
cap.release()
cv2.destroyAllWindows()

# S :: Start data collection in 100 Seconds
# P :: Pause data collection
# R :: Restart Data Collection
# Q :: Quit